In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time


In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
train_transforms = transforms.Compose([
                            transforms.Grayscale(num_output_channels=1),
                            transforms.Resize(10),
                            transforms.ToTensor()
                                      ])
train_data = datasets.ImageFolder(root = 'dga_im',transform=train_transforms)

In [4]:
train_iterator = data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size = 8)

In [5]:
mean = 0.0
for images, _ in train_iterator:
    batch_samples = images.size(0) 
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
mean = mean / len(train_iterator.dataset)

var = 0.0
for images, _ in train_iterator:
    batch_samples = images.size(0)
    images = images.view(batch_samples, images.size(1), -1)
    var += ((images - mean.unsqueeze(1))**2).sum([0,2])
std = torch.sqrt(var / (len(train_iterator.dataset)*10*10))

In [6]:
print(mean)
print(std)

tensor([0.1446])
tensor([0.1971])


In [7]:
train_transforms = transforms.Compose([
                            transforms.Grayscale(num_output_channels=1),
                            transforms.Resize(10),
                            transforms.ToTensor(),
                            transforms.Normalize(mean = [mean], std = [std])
                                      ])
train_data = datasets.ImageFolder(root = 'dga_im',transform=train_transforms)
RATIO = 0.9
n_train_examples = int(len(train_data) * RATIO)
n_orginalvalid_examples = len(train_data) - n_train_examples
n_valid_examples = int((n_orginalvalid_examples)*0.5)
n_test_examples = n_orginalvalid_examples - n_valid_examples
print(n_train_examples)
print(n_valid_examples)
print(n_test_examples)

train_data, valid_data, test_data = data.random_split(train_data, 
                                           [n_train_examples, n_valid_examples, n_test_examples])

507
28
29


In [8]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')

Number of training examples: 507
Number of validation examples: 28


In [9]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')

Number of training examples: 507
Number of validation examples: 28


In [10]:
class LeNet(nn.Module):
    def __init__(self, output_dim):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels = 1, 
                               out_channels = 3, 
                               kernel_size = 3)
        
        self.conv2 = nn.Conv2d(in_channels = 3, 
                               out_channels = 6, 
                               kernel_size = 3)
        
        self.fc_1 = nn.Linear(216, 72)
        self.fc_2 = nn.Linear(72, 24)
        self.fc_3 = nn.Linear(24, output_dim)

    def forward(self, x):

        
        
        x = self.conv1(x)
        
        
        
        
        
        
        x = F.relu(x)
        
        x = self.conv2(x)
        
        

        
        
        
        x = F.relu(x)
        
        x = x.view(x.shape[0], -1)
        
        
        
        h = x
        
        x = self.fc_1(x)
        
        
        
        x = F.relu(x)

        x = self.fc_2(x)
        
        x = F.relu(x)

        x = self.fc_3(x)
    
    

        
        
        return x, h


In [11]:
OUTPUT_DIM = 6

model = LeNet(OUTPUT_DIM)

In [12]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 17,724 trainable parameters


In [13]:
optimizer = optim.Adam(model.parameters())

In [14]:
BATCH_SIZE=8
train_iterator = data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size = BATCH_SIZE)

valid_iterator = data.DataLoader(valid_data, 
                                 batch_size = BATCH_SIZE)

In [15]:
optimizer = optim.Adam(model.parameters())

In [16]:
criterion = nn.CrossEntropyLoss()

In [17]:
device = torch.device('cpu')

In [18]:
device

device(type='cpu')

In [19]:
model = model.to(device)
criterion = criterion.to(device)

In [20]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [21]:
from tqdm import tqdm

In [22]:

def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        y_pred, _ = model(x)
        
        loss = criterion(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)

            y_pred, _ = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs





In [23]:
val_acc=[]
train_acc=[]

train_loss=[]
val_loss=[]
epoch_x_axis=[i for i in range(0,100)]

In [24]:
EPOCHS = 125
best_valid_loss = float('inf')

for epoch in tqdm(range(EPOCHS)):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut-model.pt')
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    


  1%|▋                                                                                 | 1/125 [00:01<02:43,  1.32s/it]

Epoch: 01 | Epoch Time: 0m 1s
	Train Loss: 1.738 | Train Acc: 24.74%
	 Val. Loss: 1.654 |  Val. Acc: 25.00%


  2%|█▎                                                                                | 2/125 [00:02<02:33,  1.25s/it]

Epoch: 02 | Epoch Time: 0m 1s
	Train Loss: 1.382 | Train Acc: 44.21%
	 Val. Loss: 1.289 |  Val. Acc: 46.88%


  2%|█▉                                                                                | 3/125 [00:03<02:31,  1.24s/it]

...

Epoch: 123 | Epoch Time: 0m 0s
	Train Loss: 0.222 | Train Acc: 91.99%
	 Val. Loss: 0.467 |  Val. Acc: 78.12%


 99%|███████████████████████████████████████████████████████████████████████████████▎| 124/125 [02:22<00:00,  1.02it/s]

Epoch: 124 | Epoch Time: 0m 0s
	Train Loss: 0.203 | Train Acc: 92.77%
	 Val. Loss: 0.845 |  Val. Acc: 71.88%


100%|████████████████████████████████████████████████████████████████████████████████| 125/125 [02:23<00:00,  1.15s/it]

Epoch: 125 | Epoch Time: 0m 0s
	Train Loss: 0.239 | Train Acc: 90.23%
	 Val. Loss: 0.528 |  Val. Acc: 84.38%


In [25]:
test_transforms = transforms.Compose([
                            transforms.Grayscale(num_output_channels=1),
                            transforms.Resize(10),
                            transforms.ToTensor(),
                            transforms.Normalize(mean = [mean], std = [std])
                                      ])

test_iterator = data.DataLoader(test_data, 
                                batch_size = BATCH_SIZE)
model.load_state_dict(torch.load('tut-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.479 | Test Acc: 87.50%
